# 4 - Aprendizado Profundo

O aprendizado profundo consiste em técnicas de aprendizado de máquina que usam algoritmos baseados em Redes Neurais Artificiais. Existem várias arquiteturas possíveis para essas redes. Neste notebook vamos explorar uma Rede CNN (Convolutional Neural Network) usada para classificação de imagens.

--------------------------------
## 1- Convolutional Neural Network usando Keras e TensorFlow

#### Descrição geral:
Vamos usar o dataset CIFAR 10, uma coleção de 60 mil imagens. As imagens são divididas em 10 classes, sendo 6000 exemplos para da classe. Ele é um subconjunto do 80 Million Tiny Images, contendo 80 mil imagens que foram rotuladas por estudantes.

#### Objetivo:
Neste exemplo, queremos criar um sistema de classificação automático para imagens, onde dada uma imagem é uma entrada que é classificada em uma das 10 classes.


#### Features (variáveis de entrada):
- Imagem: matriz de 32x32, RGB (3 canais de cores)

#### Alvo (valor de saída):
- Classe da imagem: 
    *   Aviões
    *   Carros
    *   Pássaros
    *   Gatos
    *   Veados
    *   Cachorros
    *   Sapos
    *   Cavalos
    *   Návios
    *   Caminhões

#### Referências:
- https://paperswithcode.com/dataset/cifar-10 
- https://en.wikipedia.org/wiki/CIFAR-10
- Algumas das arquiteturas de rede aqui apresentadas foram adaptadas de exemplos de aula do professor Miguel Bozer.

In [ ]:
# Bibliotecas gerais
import numpy as np                     # Trabalhar com números
import pandas as pd                    # Trabalhar com tabelas
import matplotlib.pyplot as plt        # Gráficos

# Bibliotecas para construir redes neurais
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from keras.utils.vis_utils import plot_model

# Aprendizado de máquina geral - Métrica de Desempenho (teste)
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report

### 1 - Passo 1 e 2: carregando os dados e separando os dados

O Keras possui uma API para carregar o Cifar-10 por se tratar de um dataset amplamente conhecido e usado. Perceba que ao carregar usando a biblioteca, já estamos separando as imagens em treino e teste. 

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

Plotando algumas das imagens:

In [ ]:
for i in range(9):  
    plt.subplot(330 + 1 + i) # define subplot
    plt.imshow(train_images[i]) # plot raw pixel data
# show the figure
plt.show()

Temos um total de 60000 imagens (50k para treino, 10k para teste) diferentes com dimensão de 32x32x3 pixels:

In [ ]:
len(train_images), len(train_labels), len(test_images), len(test_labels)

In [ ]:
train_images.shape

### 1 - Passo 3: trasnformação dos rótulos e imagens

Vamos transformar os rótulos que estão em arrays numéricos em uma matriz de categorias.

Ver https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical.

In [ ]:
train_labels

In [ ]:
train_labels = tf.keras.utils.to_categorical(train_labels)
train_labels.shape

In [ ]:
train_labels

Cada imagem é uma matriz de números (no caso RGB, três matrizes). Cada posição da matriz é chamada de pixel e possui um valor de intensidade. Em uma imagem RGB 8 bits, a intesidade de cada cor tem um valor inteiro entre 0 e 255 ($2^8$ - ou seja, 8 bits de codificação por valor). Vamos normalizar esse valor para ficar entre 0 e 1. Ao dividir a matriz por um mesmo número, estamos divindo o valor de cada pixel por esse número.

In [ ]:
# Normalizando os valores dos pixel para serem entre 0 e 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### 1 - Passo 4': Criando a Arquitetura da Rede Neural Artificial

Neste passo queremos construir nosso algoritmo de Aprendizado de Máquina Supervisionado para classificar as imagens. De vez de usar um algoritmo pronto, iremos projetar o nosso próprio seguindo uma receita conhecida: Redes Neurais Artificias do tipo CNNs.

Como queremos realizar o processamento de imagens, podemos usar **redes neurais convolucionais (CNN)**. Esse tipo de rede tem seus parâmetros em uma matriz (*kernel*) que processa toda a imagem passando por todos os seus pixels. Os elementos dessa matriz são exatamente os parâmetros que serão aprendidos pela rede. A cada etapa, ocorrerá um procedimento de multiplicação entre parte da imagem e a matriz de kernel, o que é semelhante a uma operação de convolução de sinais. Como imagens coloridas possuem três canais de cor (seguindo codificação RGB) haverá um *kernel* para cada canal.

No nosso modelo de Rede Neural, vamos conectar sequencialmente diferentes camadas com diferentes propósitos. Após a primeira camada de entrada, vamos passar por duas camadas de Convolução. Perceba que as funções de ativação das camadas de convolução é uma função tipo ReLu. Na primeira camada densa, iremos manter a função de ativação ReLu, e na última camada da rede, usaremo uma função <code>Softmax</code> já que queremos fazer classificação.

Na sequência das duas camadas de Convolução inciais, temos uma camada de <code>Max Pooling</code>. Essa camada é utilizada para fazer com que a rede seja invariante a pequenas alterações na posição da imagem nos pixels de entrada 

Temos também a camada de <code>Flatten</code> que apenas irá transformar a saída da terceira convolução (que é bidimensional) em um vetor. Isso é feito para que posteriormente seja possível implementar uma **rede neural densa** (como a rede Multilayer Perceptron).

In [ ]:
modelo = tf.keras.Sequential()

modelo.add(Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3))) # Entra com imagem 32,32,3
modelo.add(Conv2D(32, (3, 3), activation='relu')) # Segunda camada convolucional
modelo.add(MaxPooling2D((2, 2),))                 # Camada de MaxPooling
modelo.add(Conv2D(32, (3, 3), activation='relu')) # Terceira camada convolucional 
modelo.add(Flatten())                             # 
modelo.add(Dense(16, activation='relu'))          # Camada totalmente conectada
modelo.add(Dense(10, activation='softmax'))       # Camada totalmente conectada para classificação


Uma vez definida a arquitetura da rede, podemos finalizar dizendo qual será o método de otimização usado (parâmetro <code>optimizer</code>), a função de custo que será usada (parâmetro <code>loss</code>) e a métrica de desempenho que gostariamos de medir (parâmetro <code>metrics</codes>).

In [ ]:
modelo.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]) 

In [ ]:
modelo.summary() # Printar a rede construída

In [ ]:
plot_model(modelo, to_file='model_plot.png', show_shapes=True, show_layer_names=True) # Gerar uma imagem da rede

### 1 - Passo 4 - Realizando o treinamento do algoritmo

Para realizar o treinado na rede arquitetada na etapa anteriore, devemos selecionar alguns hiperparâmetros:

- <code>epochs</code>: a propagação (forward) e a retropropagação (backward) de **todos** os exemplos de treinamento;
- <code>batch_size</code>: é o número de exemplos de treino que serão passados em cada etapa de propagação e retropropagação (forward/backward). Uma época é dividida em vários **lotes** de treino até concluir a passagem sobre todos os exemplos. Um **lote** igual ao número de exemplos de treino significa que tudo será passado uma única vez, os erros de treinamento serão calculados e os pesos da rede atualizados. Isso contudo, consome muita memória RAM (para lembrar de todos os exemplos durante a época) e é mais demorado. Quebrar em lotes ajuda a acelerar e consumir menos memória RAM (quanto menores os lotes, mais rápido e menor o uso de RAM), contudo pior será a estimativa do gradiente na hora de atuliazar os pesos da rede;
- <code>validation_split</code>: significa que iremos separar 20% dos dados de treinamento para realizar uma validação ao final de cada época de treinamento. A validação consistirá em passar os dados de validação na rede recém treinado e calcular o erro. Com isso saberemos se o número de épocas escolhido está causando overfitting (é esperado que quanto maior o número de épocas, menor será o erro de treinamento - ao custo de uma menor capacidade de generalização da rede);
- <code>verbose</code>: para printar o progresso de treinamento;

Ver https://keras.io/api/models/model_training_apis/ para mais detalhes.

In [ ]:
# Vamos fazer o treinamento utilizando 25 epocas,
# Os dados de treino serão dividos em lotes de 200 exemplos (serão necessáris 200 etapas a cada época)
history = modelo.fit(train_images,train_labels,epochs=25, verbose=1, batch_size=200, validation_split=0.2)

Vamos plotar um gráfico do erro de treinamento e do erro de validação em função do número de épocas. O nosos ojetivo é ver a partir de qual número de épocas começa a ocorrer overfitting da rede.

In [ ]:
# Plotando o MSE para cada época de treinamento
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Função de Custo vs Época')
plt.ylabel('Função de Custo')
plt.xlabel('Épocas')
plt.legend(['treino', 'validação'], loc='upper right')
plt.show()

Podemos perceber que nesse primeiro treinamento houve um overfitting, uma vez que o erro de teste realizado com o conjunto de validação estabilizar a partir de 10 épocas, enquanto o erro de treinamento continua diminuindo. Isso indica que, para manter a capacidade de generalização da rede, devemos retreiná-la com apenas 10 épocas. 

### 1 - Passo 5: testar e avaliar

Nesta etapa iremos passar as 10 mil imagens de teste que o modelo treinado não viu durante o treinamento (nem a validação). Em seguida, calcularamos as métricas de desempenho mais tradicionais para a clssificação: matriz de confusão, acurácia, precisão, revocação e f1-score.

In [ ]:
y_pred = modelo.predict(test_images, batch_size=200).argmax(axis=1)

In [ ]:
y_pred

In [ ]:
test_labels

In [ ]:
cm_rn = confusion_matrix(test_labels, y_pred, labels = [0,1,2,3,4,5,6,7,8,9])

In [ ]:
figure = plt.figure(figsize=(30, 20))
disp = ConfusionMatrixDisplay(confusion_matrix = cm_rn, display_labels=[0,1,2,3,4,5,6,7,8,9])
disp.plot(values_format='d') 

In [ ]:
# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(test_labels, y_pred))

Os resultados obtidos são melhores do que um classificador de maioria (10% de chance ao acaso para acurácia), mas estão piores do que um dos primeiros artigos publicados, em 2010 cerca e 79% de acurácia. Podemos fazer melhor?

## 1 - Retornando ao Passo 4': Criando um novo modelo

Agora vamos criar um segundo modelo para tentar conseguir um resultado melhor. Vamos usar outros blocos de construição da rede: o <code>padding</code> e o <code>dropout</code>.

O <code>padding</code>  é uma forma de realizar a convolução sem alterar a dimensão da saída da operação, adicionando uma margem com zeros na imagem.

O <code>dropout</code> é uma técnica que faz com que alguns neurônios da rede sejam desativados aleatoriamente durante o treinamento. O objetivo disso é evitar o overfitting. Isso é similar a treinarmos diferentes redes (com diferentes neurônios) e obtermos a média delas como saída. No caso o dropout irá eliminar 20% dos neurônios durante o treinamento.

In [ ]:
model2 = tf.keras.Sequential()

model2.add(Conv2D(32, (3, 3), input_shape=(32,32,3), activation='relu', padding='same')) 
model2.add(Dropout(0.2)) 
model2.add(Conv2D(32, (3, 3), activation='relu', padding='same')) 
model2.add(MaxPooling2D(pool_size=(2, 2))) 
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model2.add(Dropout(0.2)) 
model2.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model2.add(MaxPooling2D(pool_size=(2, 2))) 
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model2.add(Dropout(0.2)) 
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model2.add(MaxPooling2D(pool_size=(2, 2))) 
model2.add(Flatten()) 
model2.add(Dropout(0.2)) 
model2.add(Dense(1024, activation='relu')) 
model2.add(Dropout(0.2)) 
model2.add(Dense(512, activation='relu')) 
model2.add(Dropout(0.2)) 
model2.add(Dense(10, activation='softmax'))

model2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

### 1 - Passo 4: realizando o novo treinamento:

Perceba que esse treinamento irá durar substancialmente mais do que o primeiro, uma vez que temos mais processos ocorrendo ao longo da rede neural.

In [ ]:
history_2 = model2.fit(train_images, train_labels, epochs=10, verbose=1, batch_size=200, validation_split=0.2)

In [ ]:
# Plotando o MSE para cada época de treinamento
plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title('Função de Custo vs Época')
plt.ylabel('Função de Custo')
plt.xlabel('Épocas')
plt.legend(['treino', 'validação'], loc='upper right')
plt.show()

### 1 - Passo 5: avaliando o desempenho do novo classificador

Agora, usando a porção de dados de teste, iremos realizar as previsões de todos os valores de saída, assim como fizemos com o outro modelo.

In [ ]:
y_pred = model2.predict(test_images, batch_size=200).argmax(axis=1)

In [ ]:
y_pred

In [ ]:
cm_rn = confusion_matrix(test_labels, y_pred, labels = [0,1,2,3,4,5,6,7,8,9])

In [ ]:
figure = plt.figure(figsize=(30, 20))
disp = ConfusionMatrixDisplay(confusion_matrix = cm_rn, display_labels=[0,1,2,3,4,5,6,7,8,9])
disp.plot(values_format='d') 

In [ ]:
# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(test_labels, y_pred))

### 1 - Passo 6: salvar o modelo

O modelos podem ser exportados como modelos pré-treinados no formato <code>.h5</code>, seguindo o comando abaixo:

In [ ]:
model2.save('CNN_Classificador_2.h5')  # Para salvar

In [ ]:
novo_modelo = load_model('CNN_Classificador_2.h5')  # Para carregar

------------------------
## 2 - LDL Paper

#### Descrição geral: 
Neste artigo são apresentados tutoriais introdutórios ao aprendizado profundo. São 6 scripts notebook abordando redes multiplayer perpectron, CNN, LSTM, RBM, GAN e Autoencoder.

#### Objetivo:
Entender a construção de redes neurais artificiais com arquiteturas diferentes para a solução de problemas diversos (não necessariamente da física).

#### Referências:
- Arruda, H. F. de, Benatti, A., Comin, C. H., & Costa, L. da F. (2022). Learning Deep Learning. In Revista Brasileira de Ensino de Física. FapUNIFESP (SciELO). https://doi.org/10.1590/1806-9126-rbef-2022-0101 
- https://github.com/hfarruda/deeplearningtutorial 

In [ ]:
# Executar os tutoriais no github dos autores originais

--------------------------------------------------
## Exercícios propostos

- No artigo de Cuomo et al. 2022, https://arxiv.org/pdf/2201.05624.pdf, são discutidas arquiteturas de redes neurais com modificações para serem fiscamente informadas (rede **PINN - Physics-Informed Neural Network**). Tente construir uma dessas redes usando Keras e Tensor Flow.

- Uma rede que utiliza equações diferenciais ordinárias é proposta por Chen et al. 2018, https://papers.nips.cc/paper_files/paper/2018/file/69386f6bb1dfed68692a24c8686939b9-Paper.pdf. Como construir uma NODE?

- O livro Physics-based Deep Learning apresenta vários exemplos de adaptação de redes neurais para problemas de física. Você pode encontrar esses exemplos aqui: https://physicsbaseddeeplearning.org/intro.html